# Growth Dynamics and Asset Fluctuations

## Overview
- **What this notebook does:** Explores how growth rate, operating margin, retention ratio, and insurance deductible interact to determine long-term wealth accumulation, asset volatility, ROA distributions, and survival.
- **Prerequisites:** [core/06_long_term_dynamics.ipynb](06_long_term_dynamics.ipynb)
- **Estimated runtime:** 2--5 minutes
- **Audience:** [Practitioner]

## Why Growth Dynamics Matter
Insurance optimization cannot be separated from the business it protects. Growth rate, operating margin, and earnings retention all affect how quickly a company can absorb losses and how much risk it can tolerate. This notebook quantifies those interactions.

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/core'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install git+https://github.com/AlexFiliakov/Ergodic-Insurance-Limits.git -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

from ergodic_insurance import ManufacturerConfig
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# Reproducibility
np.random.seed(42)

## Configuration

In [ ]:
INITIAL_ASSETS = 10_000_000
SIMULATION_YEARS = 100
N_PATHS = 100

# Insurance parameters for growth analysis
INSURANCE_DEDUCTIBLE = 1_000_000
INSURANCE_LIMIT = 10_000_000

# Growth scenarios to compare
GROWTH_SCENARIOS = [
    {'name': 'Conservative', 'growth_rate': 0.01, 'base_operating_margin': 0.06, 'retention_ratio': 0.8},
    {'name': 'Moderate', 'growth_rate': 0.03, 'base_operating_margin': 0.08, 'retention_ratio': 1.0},
    {'name': 'Aggressive', 'growth_rate': 0.05, 'base_operating_margin': 0.10, 'retention_ratio': 1.0},
    {'name': 'High Margin', 'growth_rate': 0.03, 'base_operating_margin': 0.15, 'retention_ratio': 0.9},
    {'name': 'Dividend Focus', 'growth_rate': 0.02, 'base_operating_margin': 0.08, 'retention_ratio': 0.5},
]

scenarios_df = pd.DataFrame(GROWTH_SCENARIOS)
print("Growth Scenarios:")
print("=" * 60)
print(scenarios_df.to_string(index=False))

## 1. Asset Path Simulation

Simulate multiple asset paths under each growth scenario with stochastic claims.

In [ ]:
def simulate_asset_paths(growth_rate, operating_margin, retention_ratio):
    """Simulate N_PATHS asset paths and return (asset_paths, roa_paths)."""
    all_assets, all_roa = [], []

    for path_id in range(N_PATHS):
        config = ManufacturerConfig(
            initial_assets=INITIAL_ASSETS,
            asset_turnover_ratio=1.0,
            base_operating_margin=operating_margin,
            tax_rate=0.25,
            retention_ratio=retention_ratio,
        )
        manufacturer = WidgetManufacturer(config)

        claim_gen = ManufacturingLossGenerator.create_simple(
            frequency=0.1, severity_mean=3_000_000, severity_std=2_000_000,
            seed=42 + path_id,
        )

        # Revenue for loss generation
        revenue = INITIAL_ASSETS * 1.0  # assets * turnover

        asset_path = [float(manufacturer.total_assets)]
        roa_path = []

        for year in range(SIMULATION_YEARS):
            # Generate losses for this year
            losses, _ = claim_gen.generate_losses(
                duration=1.0, revenue=revenue, include_catastrophic=True, time=float(year),
            )
            for loss in losses:
                manufacturer.process_insurance_claim(
                    loss.amount, INSURANCE_DEDUCTIBLE, INSURANCE_LIMIT)

            metrics = manufacturer.step(letter_of_credit_rate=0.015, growth_rate=growth_rate)
            asset_path.append(float(metrics['assets']))
            roa_path.append(float(metrics['roa']))

            if manufacturer.is_ruined:
                break

        all_assets.append(asset_path)
        all_roa.append(roa_path)

    return all_assets, all_roa

scenario_paths, scenario_roa = {}, {}
for s in GROWTH_SCENARIOS:
    print(f"Simulating {s['name']}...")
    paths, roa = simulate_asset_paths(
        s['growth_rate'], s['base_operating_margin'], s['retention_ratio'])
    scenario_paths[s['name']] = paths
    scenario_roa[s['name']] = roa

print("\nAll scenarios simulated.")

## 2. Visualize Asset Evolution

Fan charts show the median, interquartile range, and 5-95th percentile range of asset paths.

In [ ]:
n_cols = 3
n_rows = (len(GROWTH_SCENARIOS) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 5 * n_rows), squeeze=False)
axes_flat = axes.flatten()

for idx, s in enumerate(GROWTH_SCENARIOS):
    ax = axes_flat[idx]
    paths = scenario_paths[s['name']]
    max_len = max(len(p) for p in paths)
    arr = np.full((len(paths), max_len), np.nan)
    for i, p in enumerate(paths):
        arr[i, :len(p)] = p

    years = np.arange(max_len)
    median = np.nanmedian(arr, axis=0) / 1e6
    p25 = np.nanpercentile(arr, 25, axis=0) / 1e6
    p75 = np.nanpercentile(arr, 75, axis=0) / 1e6
    p5 = np.nanpercentile(arr, 5, axis=0) / 1e6
    p95 = np.nanpercentile(arr, 95, axis=0) / 1e6

    ax.fill_between(years, p5, p95, alpha=0.2, color='blue', label='5-95%')
    ax.fill_between(years, p25, p75, alpha=0.4, color='blue', label='25-75%')
    ax.plot(years, median, 'b-', lw=2, label='Median')
    ax.axhline(y=INITIAL_ASSETS / 1e6, color='red', ls='--', alpha=0.5, label='Initial')

    ax.set_xlabel('Years')
    ax.set_ylabel('Assets ($M)')
    ax.set_title(f"{s['name']}\n(g={s['growth_rate']:.1%}, m={s['base_operating_margin']:.0%}, "
                 f"r={s['retention_ratio']:.0%})")
    ax.legend(loc='upper left', fontsize=7)
    ax.set_xlim(0, SIMULATION_YEARS)
    ax.grid(True, alpha=0.3)

for idx in range(len(GROWTH_SCENARIOS), len(axes_flat)):
    fig.delaxes(axes_flat[idx])

plt.suptitle(f'Asset Evolution Under Different Growth Scenarios ({N_PATHS} Paths Each)',
             fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 3. Asset Volatility and Risk-Adjusted Returns

In [ ]:
volatility_rows = []
for s in GROWTH_SCENARIOS:
    paths = scenario_paths[s['name']]
    growth_rates = []
    for p in paths:
        for i in range(1, min(len(p), 51)):
            if p[i - 1] > 0:
                growth_rates.append(p[i] / p[i - 1] - 1)

    final_values = [p[-1] for p in paths if len(p) > 50]

    if growth_rates and final_values:
        volatility_rows.append({
            'Scenario': s['name'],
            'Mean Growth (%)': np.mean(growth_rates) * 100,
            'Volatility (%)': np.std(growth_rates) * 100,
            'Sharpe Ratio': np.mean(growth_rates) / np.std(growth_rates)
                if np.std(growth_rates) > 0 else 0,
            'Median Final ($M)': np.median(final_values) / 1e6,
        })

vol_df = pd.DataFrame(volatility_rows)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

colors = plt.cm.viridis(np.linspace(0, 1, len(vol_df)))
ax1.scatter(vol_df['Volatility (%)'], vol_df['Mean Growth (%)'], s=200, alpha=0.7, c=colors)
for _, row in vol_df.iterrows():
    ax1.annotate(row['Scenario'], (row['Volatility (%)'], row['Mean Growth (%)']),
                 xytext=(5, 5), textcoords='offset points', fontsize=9)
ax1.set_xlabel('Volatility (Annual Std Dev %)')
ax1.set_ylabel('Mean Annual Growth Rate (%)')
ax1.set_title('Risk-Return Profile')
ax1.grid(True, alpha=0.3)

bars = ax2.bar(range(len(vol_df)), vol_df['Sharpe Ratio'], color=colors)
ax2.set_xticks(range(len(vol_df)))
ax2.set_xticklabels(vol_df['Scenario'], rotation=45, ha='right')
ax2.set_ylabel('Sharpe Ratio')
ax2.set_title('Risk-Adjusted Performance')
ax2.grid(True, alpha=0.3, axis='y')
for bar, val in zip(bars, vol_df['Sharpe Ratio']):
    ax2.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
             f'{val:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("Asset Volatility Analysis:")
print("=" * 70)
print(vol_df.to_string(index=False, float_format='%.2f'))

## 4. ROA Distribution by Scenario

In [ ]:
n_cols = 3
n_rows = (len(GROWTH_SCENARIOS) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 5 * n_rows), squeeze=False)
axes_flat = axes.flatten()

roa_stats = []
for idx, s in enumerate(GROWTH_SCENARIOS):
    ax = axes_flat[idx]
    all_roa = [r * 100 for path in scenario_roa[s['name']] for r in path if not np.isnan(r)]

    if all_roa:
        mu, sigma = np.mean(all_roa), np.std(all_roa)
        ax.hist(all_roa, bins=30, density=True, alpha=0.7, color='green', edgecolor='black')
        x = np.linspace(min(all_roa), max(all_roa), 100)
        ax.plot(x, stats.norm.pdf(x, mu, sigma), 'r-', lw=2,
                label=f'N({mu:.1f}%, {sigma:.1f}%)')
        ax.legend(fontsize=8)
        roa_stats.append({
            'Scenario': s['name'], 'Mean ROA (%)': mu,
            'Std ROA (%)': sigma, 'Skewness': stats.skew(all_roa),
        })

    ax.set_xlabel('ROA (%)')
    ax.set_ylabel('Density')
    ax.set_title(s['name'])
    ax.grid(True, alpha=0.3)

for idx in range(len(GROWTH_SCENARIOS), len(axes_flat)):
    fig.delaxes(axes_flat[idx])

plt.suptitle('Return on Assets Distribution by Growth Scenario', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print(pd.DataFrame(roa_stats).to_string(index=False, float_format='%.2f'))

## 5. Operating Margin Sensitivity

How does operating margin affect 50-year survival and final asset levels?

In [ ]:
margin_range = np.linspace(0.02, 0.20, 10)
margin_rows = []
N_MARGIN_SIMS = 50
MARGIN_YEARS = 50

for margin in margin_range:
    final_assets, survived = [], 0
    for sim in range(N_MARGIN_SIMS):
        config = ManufacturerConfig(
            initial_assets=INITIAL_ASSETS, asset_turnover_ratio=1.0,
            base_operating_margin=margin, tax_rate=0.25, retention_ratio=1.0,
        )
        mfr = WidgetManufacturer(config)
        gen = ManufacturingLossGenerator.create_simple(
            frequency=0.1, severity_mean=3_000_000, severity_std=2_000_000,
            seed=42 + sim,
        )
        revenue = INITIAL_ASSETS * 1.0  # assets * turnover

        for year in range(MARGIN_YEARS):
            losses, _ = gen.generate_losses(
                duration=1.0, revenue=revenue, include_catastrophic=True, time=float(year),
            )
            for loss in losses:
                mfr.process_insurance_claim(loss.amount, INSURANCE_DEDUCTIBLE, INSURANCE_LIMIT)
            mfr.step(letter_of_credit_rate=0.015, growth_rate=0.03)
            if mfr.is_ruined:
                break

        if not mfr.is_ruined:
            survived += 1
            final_assets.append(float(mfr.total_assets))

    margin_rows.append({
        'Margin (%)': margin * 100,
        'Survival Rate (%)': survived / N_MARGIN_SIMS * 100,
        'Median Final ($M)': np.median(final_assets) / 1e6 if final_assets else 0,
    })

margin_df = pd.DataFrame(margin_rows)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(margin_df['Margin (%)'], margin_df['Survival Rate (%)'], 'b-o', lw=2, ms=8)
ax1.fill_between(margin_df['Margin (%)'], 0, margin_df['Survival Rate (%)'], alpha=0.2)
ax1.set_xlabel('Operating Margin (%)')
ax1.set_ylabel('50-Year Survival Rate (%)')
ax1.set_title('Margin vs Survival')
ax1.set_ylim(0, 105)
ax1.grid(True, alpha=0.3)

ax2.plot(margin_df['Margin (%)'], margin_df['Median Final ($M)'], 'g-s', lw=2, ms=8)
ax2.set_xlabel('Operating Margin (%)')
ax2.set_ylabel('Median Final Assets ($M)')
ax2.set_title('Margin vs Asset Growth')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(margin_df.to_string(index=False, float_format='%.1f'))

## 6. Growth Rate vs Deductible Interaction

A heatmap showing how the combination of growth rate and insurance deductible affects 50-year survival.

In [ ]:
growth_rates = [0.00, 0.02, 0.04, 0.06]
deductibles = [500_000, 1_000_000, 2_000_000, 5_000_000]
N_INTERACTION_SIMS = 30
OPERATING_MARGIN = 0.08

interaction = np.zeros((len(growth_rates), len(deductibles)))

for i, growth in enumerate(growth_rates):
    for j, ded in enumerate(deductibles):
        survivals = []
        for sim in range(N_INTERACTION_SIMS):
            config = ManufacturerConfig(
                initial_assets=INITIAL_ASSETS, asset_turnover_ratio=1.0,
                base_operating_margin=OPERATING_MARGIN, tax_rate=0.25,
                retention_ratio=1.0,
            )
            mfr = WidgetManufacturer(config)
            gen = ManufacturingLossGenerator.create_simple(
                frequency=0.1, severity_mean=3_000_000, severity_std=2_000_000,
                seed=100 + sim,
            )
            revenue = INITIAL_ASSETS * 1.0

            for year in range(MARGIN_YEARS):
                losses, _ = gen.generate_losses(
                    duration=1.0, revenue=revenue, include_catastrophic=True, time=float(year),
                )
                for loss in losses:
                    mfr.process_insurance_claim(loss.amount, ded, INSURANCE_LIMIT)
                mfr.step(letter_of_credit_rate=0.015, growth_rate=growth)
                if mfr.is_ruined:
                    break
            survivals.append(not mfr.is_ruined)
        interaction[i, j] = np.mean(survivals) * 100

fig, ax = plt.subplots(figsize=(10, 7))
im = ax.imshow(interaction, cmap='RdYlGn', aspect='auto', vmin=0, vmax=100)
ax.set_xticks(range(len(deductibles)))
ax.set_yticks(range(len(growth_rates)))
ax.set_xticklabels([f'${d / 1e6:.1f}M' for d in deductibles])
ax.set_yticklabels([f'{g:.0%}' for g in growth_rates])
ax.set_xlabel('Insurance Deductible')
ax.set_ylabel('Annual Growth Rate')
ax.set_title('Growth Rate vs Deductible: 50-Year Survival Rate (%)')
plt.colorbar(im, ax=ax, label='Survival Rate (%)')

for i in range(len(growth_rates)):
    for j in range(len(deductibles)):
        ax.text(j, i, f'{interaction[i, j]:.0f}%', ha='center', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 7. Wealth Accumulation Summary

In [ ]:
wealth_rows = []
horizons = [10, 25, 50, 100]

for s in GROWTH_SCENARIOS:
    paths = scenario_paths[s['name']]
    multiples = {}
    for h in horizons:
        vals = [p[h] / p[0] for p in paths if len(p) > h]
        multiples[f'{h}Y Multiple'] = np.median(vals) if vals else np.nan

    cagr_50 = (multiples.get('50Y Multiple', 1) ** (1 / 50) - 1) * 100 \
        if not np.isnan(multiples.get('50Y Multiple', np.nan)) else 0

    wealth_rows.append({'Scenario': s['name'], **multiples, 'CAGR 50Y (%)': cagr_50})

wealth_df = pd.DataFrame(wealth_rows)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

x = np.arange(len(wealth_df))
width = 0.25
for i, h in enumerate([10, 25, 50]):
    col = f'{h}Y Multiple'
    ax1.bar(x + i * width, wealth_df[col], width, label=f'{h} Years')

ax1.set_xticks(x + width)
ax1.set_xticklabels(wealth_df['Scenario'], rotation=45, ha='right')
ax1.set_ylabel('Wealth Multiple (Median)')
ax1.set_title('Wealth Accumulation')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

colors = plt.cm.Set3(range(len(wealth_df)))
bars = ax2.bar(range(len(wealth_df)), wealth_df['CAGR 50Y (%)'], color=colors)
ax2.set_xticks(range(len(wealth_df)))
ax2.set_xticklabels(wealth_df['Scenario'], rotation=45, ha='right')
ax2.set_ylabel('CAGR (%)')
ax2.set_title('50-Year Compound Annual Growth Rate')
ax2.grid(True, alpha=0.3, axis='y')
for bar, val in zip(bars, wealth_df['CAGR 50Y (%)']):
    ax2.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
             f'{val:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("Wealth Accumulation Analysis:")
print("=" * 70)
print(wealth_df.to_string(index=False, float_format='%.2f'))

## Key Takeaways

- Operating margin is the single most important factor for survival -- there is a critical threshold around 6%.
- Moderate growth (2-3%) delivers the best risk-adjusted returns; aggressive growth increases volatility without proportional benefit.
- Low deductibles are more valuable for high-growth companies that cannot afford interruptions to compounding.
- Full earnings retention maximizes long-term wealth, but dividend strategies reduce resilience to shocks.
- The growth rate / deductible heatmap provides a practical tool for matching insurance structure to business strategy.

## Next Steps

- **Optimize retention and limits:** [optimization/01_retention_optimization.ipynb](../optimization/01_retention_optimization.ipynb)
- **See the ergodic advantage:** [core/03_ergodic_advantage.ipynb](03_ergodic_advantage.ipynb)
- **Visualize results:** [visualization/01_basic_plots.ipynb](../visualization/01_basic_plots.ipynb)